<a href="https://colab.research.google.com/github/AmalkrishnaAS/garbage-segregation-cnn/blob/main/Copy_of_Garbage_segregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np


In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d andreasantoro/split-garbage-dataset

 84% 69.0M/81.7M [00:01<00:00, 62.0MB/s]
100% 81.7M/81.7M [00:01<00:00, 65.8MB/s]


In [ ]:
!unzip  split-garbage-dataset.zip

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.4,
                                   zoom_range = 0.4,
                                   vertical_flip=True,
                                   horizontal_flip = True,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1)
training_set = train_datagen.flow_from_directory('/content/split-garbage-dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1768 images belonging to 6 classes.


In [ ]:
training_set.class_indices

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [ ]:
index=np.random.randint(len(test_set))

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/split-garbage-dataset/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 431 images belonging to 6 classes.


# Architecture(Alex-Net)

In [ ]:
cnn=tf.keras.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(input_shape=(64,64,3),filters=16,activation='relu',kernel_size=3))
cnn.add(tf.keras.layers.BatchNormalization())

In [ ]:
cnn.add(tf.keras.layers.Conv2D(padding='same',filters=16,activation='relu',kernel_size=3))
cnn.add(tf.keras.layers.BatchNormalization())

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,activation='relu',kernel_size=3))
cnn.add(tf.keras.layers.BatchNormalization())

In [ ]:
cnn.add(tf.keras.layers.Conv2D(padding='same',filters=32,activation='relu',kernel_size=3))
cnn.add(tf.keras.layers.BatchNormalization())

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64,activation='relu',kernel_size=3))
cnn.add(tf.keras.layers.BatchNormalization())

In [ ]:
cnn.add(tf.keras.layers.Conv2D(padding='same',filters=64,activation='relu',kernel_size=3))
cnn.add(tf.keras.layers.BatchNormalization())
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(2048,activation='relu'))
# cnn.add(tf.keras.layers.Dropout(0.1))
cnn.add(tf.keras.layers.Dense(2048,activation='relu'))
# cnn.add(tf.keras.layers.Dropout(0.1))
# cnn.add(tf.keras.layers.Dense(100,activation='relu'))
# cnn.add(tf.keras.layers.Dense(30,activation='relu'))
cnn.add(tf.keras.layers.Dense(6,activation='softmax'))
cnn.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(),metrics=['accuracy'])


In [ ]:
r=cnn.fit(training_set,validation_data=test_set,epochs=150)

In [ ]:
!mkdir -p saved_model
cnn.save('saved_model/my_model') 

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
!zip -r /content/best_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/my_model/ (stored 0%)
  adding: content/saved_model/my_model/saved_model.pb (deflated 88%)
  adding: content/saved_model/my_model/assets/ (stored 0%)
  adding: content/saved_model/my_model/variables/ (stored 0%)
  adding: content/saved_model/my_model/variables/variables.index (deflated 65%)
  adding: content/saved_model/my_model/variables/variables.data-00000-of-00001 (deflated 14%)
  adding: content/saved_model/my_model/keras_metadata.pb (deflated 91%)


In [ ]:
from google.colab import files
files.download('/content/best_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.preprocessing import image
test_image = image.load_img('plastic.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image/255.0)
np.argmax(result),training_set.class_indices

(4,
 {'cardboard': 0,
  'glass': 1,
  'metal': 2,
  'paper': 3,
  'plastic': 4,
  'trash': 5})